In [ ]:
import pandas as pd
from openai import OpenAI
import re
import unicodedata
from referentiels import referentiels
import numpy as np
import json
from dotenv import load_dotenv
import os

# Charger les variables depuis le fichier .env
load_dotenv()

# Récupérer la clé
api_key = os.getenv('OPENAI_API_KEY')

# Clé API OpenAI

client = OpenAI(api_key = api_key)


def load_data(filepath="data/besoins_24_25.csv"):
    import os
    import pandas as pd

    try:
        if not os.path.exists(filepath):
            raise FileNotFoundError(f"Le fichier {filepath} n'existe pas")
        df = pd.read_csv(filepath, encoding="utf-8", sep=",")
        if df.empty:
            raise pd.errors.EmptyDataError("Le fichier CSV est vide")
        return df
    except UnicodeDecodeError:
        raise ValueError(f"Erreur d'encodage du fichier {filepath}")
    except pd.errors.ParserError:
        raise ValueError(f"Format de fichier invalide: {filepath}")

# ==================== UTILS ====================

def nettoyage_general(df):
    """
    Nettoyage standard du DataFrame
    - Suppression des doublons
    - Suppression des espaces inutiles dans les colonnes texte
    - Nettoyage des points de suspension
    - Conversion des types (si possible)
    - Uniformisation des NaN
    """
    print("🔎 Nettoyage général en cours...")

    # 1. Retrait des doublons
    df = df.drop_duplicates()

    # 2. Nettoyage des colonnes texte
    for col in df.select_dtypes(include='object').columns:
        df[col] = (
            df[col]
            .astype(str)
            .str.strip()
            .str.replace(r'\.{3,}', '', regex=True)  # suppression des "....."
            .replace('', np.nan)  # remplace les vides par NaN après nettoyage
            .str.replace(r"[^\w\sÀ-ÿ',-]", "", regex=True)  # suppression caractères spéciaux
        )


    # 3. Conversion numérique si possible
    for col in df.columns:
        try:
            df[col] = pd.to_numeric(df[col])
        except (ValueError, TypeError):
            pass  # colonne non convertible

    # 4. Uniformisation des valeurs manquantes
    df.replace(['None', 'nan', 'NaN', 'N/A', '', ' '], np.nan, inplace=True)

    print("✅ Nettoyage général terminé")
    return df


def normalize(text):
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode().lower().strip()

def nettoyer_valeur(val):
    val = val.strip()
    val = re.sub(r'\s+', ' ', val)
    return val

def clean_theme(theme):
    theme = re.sub(r'^Aucun thème.*?["“”]', '', theme)
    theme = re.sub(r'["“”]', '', theme)
    return theme.strip().capitalize()

def theme_existe(theme, referentiel):
    theme_norm = normalize(theme)
    for officiel in referentiel:
        if normalize(officiel) == theme_norm:
            return officiel
    return None

# ==================== CLASSIFICATION GPT ====================

def classifier_avec_gpt(val, col, referentiels):
    liste = referentiels.get(col, [])

    prompt = f"""
Tu travailles pour le réseau OSUI - Mission Laïque Française.

Voici les réponses officielles possibles pour la question "{col}" :
{chr(10).join(f"- {v}" for v in liste) if liste else "(aucune liste connue)"}

Voici la réponse libre d'un enseignant : "{val}"

Ta tâche :
1. Si cette réponse correspond à un des thèmes officiels (même si l'orthographe, les majuscules ou l'ordre des mots sont différents), retourne exactement ce thème officiel.
2. Sinon, propose un nouveau thème court et pertinent.
3. Si la réponse est vide, hors sujet ou incompréhensible, retourne : **Autre (à clarifier)**

Ne donne qu'un seul thème. Aucune explication.
"""

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Tu es un expert en classification pédagogique."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.1
    )

    theme_propose = response.choices[0].message.content.strip()
    theme_propre = clean_theme(theme_propose)

    officiel = theme_existe(theme_propre, liste)
    if officiel:
        return officiel

    INTERDITS = ["nouveau theme", "aucun theme", "veuillez fournir", "n/a", "non renseigné", "rien"]
    if any(bad in normalize(theme_propre) for bad in INTERDITS):
        return "Autre (à clarifier)"

    return theme_propre

# ==================== MAPPING ====================

def appliquer_mapping(text, mapping):
    if pd.isna(text):
        return ""
    result = set()
    for val in text.split(','):
        val = nettoyer_valeur(val)
        if val in mapping:
            result.add(mapping[val])
        elif normalize(val) in [normalize(v) for v in mapping]:
            for k in mapping:
                if normalize(k) == normalize(val):
                    result.add(mapping[k])
        else:
            result.add(val)
    return ",".join(result)

# ==================== TRAITEMENT ====================

def reponses_hors_referentiel(df, col, referentiels):
    hors_ref = set()
    referentiel_normalise = {normalize(r) for r in referentiels}
    for ligne in df[col].dropna().astype(str):
        for val in ligne.split(','):
            val = nettoyer_valeur(val)
            if val and normalize(val) not in referentiel_normalise:
                hors_ref.add(val)
    return hors_ref


def traiter_colonne_multichoix(df, col, referentiels):
    print(f"\n🔧 Traitement : {col}")
    valeurs = reponses_hors_referentiel(df, col, referentiels[col])
    print(f"🧹 {len(valeurs)} réponses hors référentiel détectées")

    themes_col = {}

    for val in sorted(valeurs):
        theme = classifier_avec_gpt(val, col, referentiels)
        print(f"➡️ {val} → {theme}")
        themes_col[val] = theme

    nouveaux = set(themes_col.values())
    for t in nouveaux:
        if not theme_existe(t, referentiels[col]) and t != "Autre (à clarifier)":
            referentiels[col].append(t)

    df[col + '_themes'] = df[col].apply(lambda x: appliquer_mapping(x, themes_col))

    dummies = df[col + '_themes'].str.get_dummies(sep=',')
    dummies.columns = [f"{col}_theme_{c.strip().lower().replace(' ', '_')}" for c in dummies.columns]

    print(f"✅ {len(themes_col)} nouvelles valeurs mappées pour {col}")
    return df, dummies, themes_col

# ==================== SAUVEGARDE ====================

def sauvegarder_referentiel(referentiels, path="referentiels.py"):
    with open(path, "w", encoding="utf-8") as f:
        f.write("# Référentiel enrichi automatiquement\n")
        f.write("referentiels = ")
        json.dump(referentiels, f, ensure_ascii=False, indent=4)
    print(f"\n✅ Référentiel sauvegardé dans : {path}")


In [ ]:
# ==================== CONFIG ====================
# Liste des colonnes à traiter
multi_cols = [
    'certifications_educatives',
    'responsabilites_complementaires',
    'matieres_niveaux_actuels',
    'objectifs_outils_numeriques',
    'competences_dev_formations',
    'freins_formation_continue',
    'contenus_autoformation',
    'evaluations_alternatives'
]

# ==================== PIPELINE ====================
def run_pipeline(filepath, new_column_names, referentiels, multi_cols):
    """
    Pipeline complet de traitement :
    - Chargement des données
    - Renommage des colonnes
    - Nettoyage général
    - Traitement multichoix
    - Sauvegarde du référentiel enrichi et du dataframe propre
    """
    print("\n🚀 Démarrage du pipeline de traitement...\n")

    # Chargement
    df = load_data(filepath)
    df.rename(columns=new_column_names, inplace=True)

    # Nettoyage
    df = nettoyage_general(df)

    # Traitement multi-choix
    all_dummies = []
    all_themes = {}
    for col in multi_cols:
        df, dummies, themes = traiter_colonne_multichoix(df, col, referentiels)
        all_dummies.append(dummies)
        all_themes[col] = themes

    # Fusion
    df = pd.concat([df] + all_dummies, axis=1)

    # 🔥 Nettoyage final
    df = nettoyage_general(df)

    # Sauvegarde
    df.to_csv("data/cleaned_data.csv", index=False)
    sauvegarder_referentiel(referentiels, path="referentiels.py")

    print("\n✅ Pipeline terminé avec succès.")
    return df, all_themes


In [18]:
from referentiels import referentiels  # ton fichier enrichi
from new_column_names import new_column_names  # ton fichier de renommage

# Paramètres
filepath = "data/besoins_24_25.csv"

# Lancement du pipeline
df_clean, all_themes = run_pipeline(filepath, new_column_names, referentiels, multi_cols)



🚀 Démarrage du pipeline de traitement...

🔎 Nettoyage général en cours...
✅ Nettoyage général terminé

🔧 Traitement : certifications_educatives
🧹 40 réponses hors référentiel détectées
➡️ BAFA → Aucune certification déclarée
➡️ BNNSA → **autre (à clarifier)**
➡️ Brevet d'état d'éducateur sportif → Diplôme sportif
➡️ C2i2e enseignant → **autre (à clarifier)**
➡️ CAPES → Aucune certification déclarée
➡️ CPS Maroc → **autre (à clarifier)**
➡️ Certificat FLE Master → FLE
➡️ Certification de praticienne en psychothérapie → **autre (à clarifier)**
➡️ DAEFLE diplôme d'aptitudes à l'enseignement du FLE Alliance française → FLE
➡️ DESA → **autre (à clarifier)**
➡️ DU → Diplôme universitaire (DU)
➡️ Diplome de formateur PSC secourisme → Secourisme
➡️ Diplôme national marocain → Aucune certification déclarée
➡️ Discipline positive → **autre (à clarifier)**
➡️ Enseignant de langue → Enseignement des langues
➡️ Formation d'éducateurs de jeunes enfants → **autre (à clarifier)**
➡️ Formation de form